# 🏋️ SenecaCoach Prototype V2 – Flujo de trabajo

Este notebook construye un prototipo de asistente digital enfocado en **salud y entrenamiento deportivo**.  
Integra múltiples fuentes de datos (documentos médicos, datos de Strava, consultas del usuario) y las procesa usando modelos de lenguaje (LLMs) y sistemas de recuperación de información (RAG).  

El resultado final es un **plan de entrenamiento personalizado**, acompañado de un **planificador de rutas deportivas** que permite crear recorridos en función del tipo de ejercicio y de los objetivos de cada sesión. Todo esto es accesible desde una interfaz web amigable.

---

## 🔄 Flujo de trabajo paso a paso

### 1. Librerías principales
Se cargan librerías de **manejo de datos, APIs y visualización**:
- Python estándar (`os`, `re`, `json`, etc.).
- **Pandas/Numpy** para análisis de datos.
- **Matplotlib** para visualizaciones.

### 2. Procesamiento de archivos
- Manejo de datos de **reportes médicos** y **archivos GPX/Strava**.
- Uso de **OCR** para extraer texto de documentos médicos en PDF o imagen.

### 3. Interacción con LLM y APIs
- Se conecta a un **modelo de lenguaje (Groq API)** para generar respuestas y planes de entrenamiento.
- Se construye un **prompt estructurado** con delimitadores especiales (CSV) para extraer tablas fácilmente.

### 4. Recuperación de información (RAG)
- Implementación de un sistema **RAG (Retrieval-Augmented Generation)**:
  - **TF-IDF** para indexar y buscar documentos relevantes.
  - Se pasa el contexto recuperado al LLM para generar respuestas más precisas.

### 5. Plan de entrenamiento deportivo
- Con la información médica y datos de actividad física:
  - Se generan **pautas de salud**.
  - Se construye un **plan de entrenamiento** adaptado al usuario.

### 6. Interfaz web con Gradio
- Se implementa una interfaz **interactiva** para:
  - Cargar reportes médicos.
  - Procesar datos de Strava.
  - Generar y visualizar el plan de entrenamiento.

### 7. Geolocalización y visualización
- Se usan herramientas de mapas (**Folium, OSRM, Nominatim**) para:
  - Localizar puntos de interés.
  - Calcular rutas.
  - Visualizar entrenamientos en un mapa.

---

## 🛠️ Herramientas y APIs utilizadas

### 🔧 Librerías
- **pandas / numpy** → Análisis y manipulación de datos.
- **matplotlib** → Visualización de gráficos.
- **re / json / os** → Procesamiento de texto y archivos.
- **gpxpy** → Manejo de datos GPX.
- **folium** → Mapas interactivos.

### 🤖 Inteligencia Artificial
- **Groq API (LLM)** → Generación de texto y planes de entrenamiento.
- **TF-IDF (scikit-learn)** → Búsqueda y recuperación de documentos relevantes.
- **OCR (pytesseract / similares)** → Extracción de texto de imágenes o PDFs médicos.

### 🌐 APIs externas
- **Nominatim (OpenStreetMap)** → Geocodificación (nombre → coordenadas).
- **OSRM (Open Source Routing Machine)** → Rutas y distancias.
- **Open-Elevation API** → Perfil de altitud de las rutas.

### 💻 Interfaz
- **Gradio** → Construcción de una aplicación web simple e interactiva para usar el prototipo.


In [24]:
!sudo apt install tesseract-ocr
!pip install Pillow openai groq scikit-learn PyPDF2 python-docx pandas openpyxl gradio pytesseract -q

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [25]:
!pip install gradio requests folium gpxpy

In [26]:
# Librerías

# Core Python libraries
import os
import json
import csv
import io
import re
import math
import tempfile

# Data handling and analysis
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# File processing
from PIL import Image
import pytesseract
from PyPDF2 import PdfReader
from docx import Document
import openpyxl # Added openpyxl for Excel file processing

# LLM and API interaction
import openai
from groq import Groq
from google.colab import userdata

# Gradio and web interaction
import gradio as gr
import requests
from io import BytesIO

# Mapping and Geolocation
import folium
import gpxpy.gpx

# Visualization
import matplotlib.pyplot as plt

In [27]:
# RAG SIMPLE CON GROQ API
# Sistema de búsqueda en documentos usando TF-IDF y Groq para generación

# Configuración del sistema
GROQ_API_KEY = userdata.get('GROQ_KEY')
client = Groq(api_key=GROQ_API_KEY)
folder_path = '/content/carpeta_rag'

# Variables globales del sistema
vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2), min_df=2)
chunks = []
vectors = None

def load_documents(folder_path):
    """Carga y procesa documentos de múltiples formatos"""
    text = ''
    os.makedirs(folder_path, exist_ok=True)

    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)

        try:
            if filename.endswith('.txt'):
                with open(filepath, 'r', encoding='utf-8') as f:
                    text += f.read() + '\n'

            elif filename.endswith('.pdf'):
                reader = PdfReader(filepath)
                for page in reader.pages:
                    text += page.extract_text() + '\n'

            elif filename.endswith('.docx'):
                doc = Document(filepath)
                for paragraph in doc.paragraphs:
                    text += paragraph.text + '\n'

            elif filename.endswith('.csv'):
                df = pd.read_csv(filepath)
                text += df.to_string() + '\n'

            elif filename.endswith('.xlsx'):
                df = pd.read_excel(filepath)
                text += df.to_string() + '\n'
        except:
            continue

    return text

def create_chunks(text, chunk_size=500):
    """Divide el texto en fragmentos de tamaño manejable"""
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i + chunk_size])
        if len(chunk.strip()) > 100:
            chunks.append(chunk.strip())

    return chunks

def search_relevant_chunks(query, top_k=3):
    """Encuentra fragmentos más relevantes usando similitud coseno"""
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, vectors)[0]

    # Obtener índices ordenados por relevancia
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    relevant_chunks = []
    for idx in top_indices:
        if similarities[idx] > 0.1:  # Umbral mínimo de relevancia
            relevant_chunks.append(chunks[idx])

    return relevant_chunks

def generate_answer(query, context):
    """Genera respuesta usando Groq API con el contexto encontrado"""
    if not context:
        return "No encontré información relevante en los documentos."

    prompt = f"""Basándote en esta información:

    {context}

    Pregunta: {query}

    Instrucciones:
    - Responde solo con información del contexto
    - Si no hay información suficiente, di que no puedes responder
    - Responde en español de forma clara y concisa"""

    try:
        response = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=2500,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generando respuesta: {e}"

def inicializar_rag():
    """Inicializa el sistema RAG cargando documentos y creando índices"""
    global chunks, vectors, vectorizer

    documents = load_documents(folder_path)

    if not documents.strip():
        print("No se encontraron documentos en la carpeta.")
        return False

    chunks = create_chunks(documents)

    vectors = vectorizer.fit_transform(chunks)

    return True


In [28]:
# RAG para pauts de entrenamiento deportivo y salud
class HealthRAGSystem:
    def retrieve_guidelines(self, health_goal, medical_text, age, gender):

        """
        Retrieves health guidelines based on the user's goal and report.
        In a real app, this would query a vector database.
        """
        # Inicialización del sistema
        inicializar_rag()

        # BLOQUE DE INFERENCIA - Ejecutar por separado
        query = f"""Identifica las pautas de salud y actividad física para una persona con el objetivo descrito en {health_goal},
        la edad proporcionada en {age}, el género seleccionado en {gender}
        y la información médica suministrada en {medical_text}"""
        context = '\n\n'.join(search_relevant_chunks(query))
        guidelines = client.chat.completions.create(model="openai/gpt-oss-20b", messages=[{"role": "user", "content": f"Contexto: {context}\nPregunta: {query}\nResponde solo con información del contexto en español."}], max_tokens=400).choices[0].message.content

        return guidelines



In [29]:
# OCR para procesamiento de información médica
def process_medical_report(image_path):
    """
    Uses Tesseract to perform OCR on a medical report image.
    This is a conceptual function.
    """
    try:
        text = pytesseract.image_to_string(image_path)
        return text
    except Exception as e:
        print(f"Error during OCR: {e}")
        return ""

In [30]:
# Plan de entrenamiento deportivo
import tempfile

def generate_training_plan(medical_text, strava_data, health_goal, duration, rag_system, age, gender):
    """
    Usa el LLM y el sistema RAG para crear un plan de entrenamiento personalizado
    y también generar un archivo CSV del plan semanal.
    """
    # Query the mock RAG system
    guidelines = rag_system.retrieve_guidelines(health_goal, medical_text, age, gender)

    # Construye el prompt para el LLM. Hemos añadido marcadores
    # ---CSV_START--- y ---CSV_END--- para que sea más fácil extraer la tabla.
    prompt = f"""

    Eres un entrenador físico impulsado por IA, especializado en entrenamiento basado en evidencia.

    Tu misión es diseñar un plan de entrenamiento para un usuario con el siguiente perfil y objetivo de actividad física:

    Su objetivo es: {health_goal}

    Edad: {age}

    Género: {gender}

    Información médica relevante: {medical_text}

    Cuentas con el siguiente resumen de su actividad física reciente para determinar su condición física actual. Si el reporte tiene
    fechas, puedes usarla para validar que se trate de información reciente:

    Resumen de actividad deportiva reciente: {strava_data}

    Con esta información, genera un plan de entrenamiento seguro y efectivo, basado en las siguientes pautas de salud de expertos:

    Pautas de salud de expertos: {guidelines}

    La duración del plan debe coincidir con la duración deseada por el usuario:

    Duración deseada por el usuario: {duration}


    --Principios de salud y plan de entrenamiento--
    Resume brevemente los principios de salud considerados y proporciona un plan semanal detallado.

    ---Plan Semanal---
    Genera el plan semanal en una tabla con la siguiente estructura:
    Semana, Lunes, Martes, Miércoles, Jueves, Viernes, Sábado, Domingo, Comentarios
    Para cada día, indica la actividad recomendada y la intensidad (Ejemplo: 'Correr 2k, intensidad baja').
    En la columna de comentarios, explica el objetivo de la semana. Cada fila debe corresponder a una semana del plan.

    ---Recomendaciones---
    Termina el plan con recomendaciones generales de Nutrición, Hidratación y Descanso, explicando los puntos clave de cada una.
    """

    # Llama al LLM (asegúrate de que el 'client' esté definido en tu entorno)
    try:
        response = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=2000,
            temperature=0.3
        )
        llm_output = response.choices[0].message.content

        # Separar el texto del CSV
        # Usamos expresiones regulares para encontrar el texto entre los marcadores
        csv_match = re.search(r'---Plan Semanal---(.*?)---Recomendaciones---', llm_output, re.DOTALL)

        csv_filepath = None

        if csv_match:
            # Elimina espacios en blanco y saltos de línea extra del CSV
            csv_content = csv_match.group(1).strip()
            # Creamos un archivo temporal para el CSV
            with tempfile.NamedTemporaryFile(mode='w', suffix=".csv", delete=False) as temp_csv_file:
                writer = csv.writer(temp_csv_file)
                # Escribimos los datos del LLM en el archivo CSV
                reader = csv.reader(io.StringIO(csv_content))
                for row in reader:
                    writer.writerow(row)

                csv_filepath = temp_csv_file.name

        # Retorna una tupla con el texto y la ruta del archivo CSV temporal.
        return llm_output, csv_filepath

    except Exception as e:
        return f"Error al procesar la consulta: {str(e)}", None

In [31]:
# The main function to connect the Gradio interface to the backend logic.

def generate_plan(medical_report_image, strava_data_file, health_goal, duration, age, gender):
    # 1. Process medical report
    medical_text = process_medical_report(medical_report_image)
    if not medical_report_image or not medical_text:
        return "Por favor sube una imagen clara del reporte médico.", None

    # 2. Process Strava data (simplified for this example)
    strava_data = "No has suministrado tu resumen de actividad física."
    if strava_data_file:
        try:
            # In a real app, you would parse the data properly
            strava_data = f"Resumen de actividad física recibido: {strava_data_file.name}"
        except Exception:
            return "No he podido procesar el resumen de actividad física, asegúrate de que esté en el formato correcto.", None

    # 3. Initialize RAG system
    rag_system = HealthRAGSystem()

    # 4. Generate the training plan
    training_plan_text, csv_filepath = generate_training_plan(
        medical_text=medical_text,
        strava_data=strava_data,
        health_goal=health_goal,
        duration=duration,
        rag_system=rag_system,
        age=age,
        gender=gender
    )

    return training_plan_text, csv_filepath

In [32]:
# ==========================
# 1. Geocodificación con Nominatim
# ==========================
def geocode_place(place_name):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": place_name, "format": "json", "limit": 1}
    headers = {"User-Agent": "StravaTrainingApp/1.0"}
    resp = requests.get(url, params=params, headers=headers)
    data = resp.json()
    if len(data) == 0:
        raise ValueError(f"No se encontró ubicación para: {place_name}")
    lat = float(data[0]["lat"])
    lon = float(data[0]["lon"])
    return lat, lon

# ==========================
# 2. Distancia con OSRM
# ==========================
def get_route_distance(start_lat, start_lon, end_lat, end_lon, modo="foot"):
    url = f"http://router.project-osrm.org/route/v1/{modo}/{start_lon},{start_lat};{end_lon},{end_lat}"
    resp = requests.get(url, params={"overview": "false"})
    data = resp.json()
    dist_km = data["routes"][0]["distance"] / 1000
    return dist_km

# ==========================
# 3. Elevación con Open-Elevation
# ==========================
def get_elevations(coords, batch_size=50):
    elevations = []
    for i in range(0, len(coords), batch_size):
        batch = coords[i:i+batch_size]
        locations = "|".join([f"{lat},{lon}" for lon, lat in batch])
        url = f"https://api.open-elevation.com/api/v1/lookup?locations={locations}"
        resp = requests.get(url)
        data = resp.json()
        if "results" not in data:
            batch_elev = [0] * len(batch)
        else:
            batch_elev = [r["elevation"] for r in data["results"]]
        elevations.extend(batch_elev)
    return elevations

def calc_desnivel(elevations):
    return sum(max(0, elevations[i+1] - elevations[i]) for i in range(len(elevations)-1))

# ==========================
# 3b. Suavizado
# ==========================
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

def smooth_elevations_auto(elevations, coords, distancia_km):
    dists = [0]
    for i in range(1, len(coords)):
        lon1, lat1 = coords[i-1]
        lon2, lat2 = coords[i]
        d = haversine(lat1, lon1, lat2, lon2)
        dists.append(dists[-1] + d)
    total_km = dists[-1]
    n_points = len(coords)
    spacing_m = (total_km * 1000) / n_points

    target_smooth_m = distancia_km * 15
    window = max(3, int(target_smooth_m / spacing_m))

    pad_width = window // 2
    padded = np.pad(elevations, pad_width, mode="reflect")

    elev_smooth = np.convolve(padded, np.ones(window)/window, mode="same")
    elev_smooth = elev_smooth[pad_width:-pad_width]

    return elev_smooth

# ==========================
# 3c. Gráfica de perfil
# ==========================

def save_elevation_plot(coords, elevations):
    """Genera y guarda el perfil de elevación en un archivo temporal PNG."""
    dists = [0]
    for i in range(1, len(coords)):
        lon1, lat1 = coords[i-1]
        lon2, lat2 = coords[i]
        d = haversine(lat1, lon1, lat2, lon2)
        dists.append(dists[-1] + d)

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(dists, elevations, color="blue", linewidth=1.5)
    ax.fill_between(dists, elevations, alpha=0.3, color="skyblue")
    ax.set_title("Perfil de elevación")
    ax.set_xlabel("Distancia (km)")
    ax.set_ylabel("Elevación (m)")
    ax.set_ylim(min(elevations)*0.9, max(elevations)*1.1)
    ax.grid(True, alpha=0.3)

    # Guardar en archivo temporal
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    fig.savefig(tmp_file.name, bbox_inches="tight")
    plt.close(fig)  # cerrar la figura para liberar memoria
    return tmp_file.name


# ==========================
# 3d. Guardar GPX
# ==========================
def save_gpx(coords, elevations, output_file="ruta.gpx"):
    gpx = gpxpy.gpx.GPX()
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)
    for (lon, lat), ele in zip(coords, elevations):
        gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(lat, lon, elevation=float(ele)))
    with open(output_file, "w") as f:
        f.write(gpx.to_xml())
    return output_file

# ==========================
# 4. Direcciones cardinales
# ==========================
DIRECCIONES = {
    "N":  (1, 0), "S":  (-1, 0), "E":  (0, 1), "W":  (0, -1),
    "NE": (1, 1), "NW": (1, -1), "SE": (-1, 1), "SW": (-1, -1),
}

# ==========================
# 5. Generar ruta circular
# ==========================
def generar_ruta_loop(start_lat, start_lon, distancia_km=0, velocidad_kmh=10, tiempo_min=60,
                      modo="foot", direccion="N", max_iter=25):

    if distancia_km == 0:
        distancia_km = int(velocidad_kmh * tiempo_min / 60)

    dx, dy = DIRECCIONES[direccion.upper()]
    radio = distancia_km / 2.0

    for _ in range(max_iter):
        dlat = (dx * radio) / 111.32
        dlon = (dy * radio) / (111.32 * math.cos(math.radians(start_lat)))
        waypoint_lat = start_lat + dlat
        waypoint_lon = start_lon + dlon
        dist_ida = get_route_distance(start_lat, start_lon, waypoint_lat, waypoint_lon, modo)
        dist_vuelta = get_route_distance(waypoint_lat, waypoint_lon, start_lat, start_lon, modo)
        dist_total = dist_ida + dist_vuelta
        error = dist_total - distancia_km
        if abs(error) < distancia_km * 0.1:
            break
        radio *= distancia_km / dist_total

    # Obtener ruta
    url_ida = f"http://router.project-osrm.org/route/v1/{modo}/{start_lon},{start_lat};{waypoint_lon},{waypoint_lat}"
    resp_ida = requests.get(url_ida, params={"overview": "full", "geometries": "geojson"})
    coords_ida = resp_ida.json()["routes"][0]["geometry"]["coordinates"]

    url_vuelta = f"http://router.project-osrm.org/route/v1/{modo}/{waypoint_lon},{waypoint_lat};{start_lon},{start_lat}"
    resp_vuelta = requests.get(url_vuelta, params={"overview": "full", "geometries": "geojson"})
    coords_vuelta = resp_vuelta.json()["routes"][0]["geometry"]["coordinates"]

    coords = coords_ida + coords_vuelta

    # Elevaciones suavizadas
    elevations_raw = get_elevations(coords)
    elevations = smooth_elevations_auto(elevations_raw, coords, distancia_km)
    desnivel = calc_desnivel(elevations)

    # Archivos de salida
    gpx_file = save_gpx(coords, elevations, f"ruta_loop_{direccion}.gpx")
    elevation_img = save_elevation_plot(coords, elevations)

    # Mapa Folium
    m = folium.Map(location=[start_lat, start_lon], zoom_start=14)
    folium.PolyLine([(lat, lon) for lon, lat in coords], color="blue", weight=4).add_to(m)
    folium.Marker([coords[0][1], coords[0][0]], tooltip="Inicio", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker([coords[-1][1], coords[-1][0]], tooltip="Fin", icon=folium.Icon(color="red")).add_to(m)

    return coords, dist_total, desnivel, gpx_file, elevation_img, m._repr_html_()


In [33]:
# ==========================
# 6. Visualizar archivo GPX
# ==========================
def visualizar_gpx(file_obj):
  try:
      with open(file_obj.name, "r", encoding="utf-8") as f:
          gpx = gpxpy.parse(f)
      if not gpx.tracks:
          return None
      first_point = gpx.tracks[0].segments[0].points[0]
      start_lat, start_lon = first_point.latitude, first_point.longitude
      mapa = folium.Map(location=[start_lat, start_lon], zoom_start=13)
      coords = []
      for track in gpx.tracks:
          for segment in track.segments:
              for point in segment.points:
                  coords.append((point.latitude, point.longitude))
      if coords:
          folium.PolyLine(coords, color="blue", weight=3).add_to(mapa)
          folium.Marker(coords[0], tooltip="Inicio", icon=folium.Icon(color="green")).add_to(mapa)
          folium.Marker(coords[-1], tooltip="Fin", icon=folium.Icon(color="red")).add_to(mapa)
      return mapa
  except Exception:
      return None


def visualize_uploaded_gpx(file_obj):
    if file_obj is None:
        return None, "Por favor, sube un archivo GPX."
    try:
        map_obj = visualizar_gpx(file_obj)
        if map_obj:
            return map_obj._repr_html_(), "Mapa generado exitosamente."
        else:
            return None, "No se pudo generar el mapa. Verifica el archivo GPX."
    except Exception as e:
        return None, f"Error al procesar el archivo: {e}"

In [34]:
# ==========================
# 7. Wrapper para Gradio
# ==========================
def generate_route_with_gradio(place_name, distancia_km, velocidad_kmh, tiempo_min, modo, direccion):
    try:
        start_lat, start_lon = geocode_place(place_name)

        if distancia_km:
            velocidad_kmh = 0
            tiempo_min = 0

        coords, dist_real, desnivel, gpx_file, elevation_img, map_html = generar_ruta_loop(
            start_lat=start_lat,
            start_lon=start_lon,
            distancia_km=distancia_km,
            velocidad_kmh=velocidad_kmh,
            tiempo_min=tiempo_min,
            modo=modo,
            direccion=direccion
        )

        distance_info = (
            f"🎯 Distancia objetivo: {distancia_km or (velocidad_kmh * tiempo_min / 60):.2f} km\n"
            f"📏 Distancia real: {dist_real:.2f} km\n"
            f"⛰️ Desnivel estimado: {desnivel:.0f} m"
        )

        return map_html, gpx_file, distance_info, elevation_img

    except ValueError as e:
        return None, None, f"Error: {e}", None

In [35]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:

    # Encabezado general
    gr.Markdown(
        """
        <div style="background: linear-gradient(to right, #5a81f6, #9973BE); color: white; padding: 2rem; text-align: center; border-radius: 1.5rem 1.5rem 0 0;">
            <h1 style="font-size: 2rem; font-weight: 800; margin-bottom: 0.5rem;">Séneca Coach</h1>
            <h2 style="font-size: 1.5rem; font-weight: 800; margin-bottom: 0.5rem;">Tu entrenador personal impulsado por IA</h2>
        </div>
        """
    )

    with gr.Tabs():
        # ===============================
        # TAB 1: PLAN DE ENTRENAMIENTO
        # ===============================
        with gr.TabItem("📋 Plan de Entrenamiento"):
            gr.Markdown(
                """
                <div style="text-align: center">
                    <p style="font-size: 1.25rem; opacity: 0.9;">Comparte algunos datos sobre ti y tu meta deportiva, y deja que Séneca diseñe un plan personalizado para alcanzar tu objetivo.</p>
                </div>
                """
            )

            with gr.Row():
                age_input = gr.Number(label="Edad")
                gender_input = gr.Radio(["Masculino", "Femenino"], label="Género")
            with gr.Row():
                image_input = gr.File(label="Sube tu información médica reciente (Imagen/PDF)")
                file_input = gr.File(label="Sube un resumen de tu actividad deportiva reciente (JSON/CSV)")
            with gr.Row():
                duration_input = gr.Number(label="Duración esperada del plan (semanas)")
            with gr.Row():
                goal_input = gr.Textbox(lines=2, label="Tu objetivo de actividad física", placeholder="Ej., Correr 5k, aumentar tu masa muscular, mejorar tu resistencia física, recuperarte de una lesión...")

            submit_button = gr.Button("Genera mi Plan",
                                       variant="primary",
                                       scale=0.5,
                                       elem_classes="w-full sm:w-auto px-8 py-3 bg-indigo-500 hover:bg-indigo-600 text-white font-bold rounded-full shadow-xl transition-all duration-300 transform hover:scale-105")

            plan_output = gr.Markdown(label="Plan de Entrenamiento")
            csv_download = gr.File(label="Descargar Plan en CSV")

            submit_button.click(
                fn=generate_plan,
                inputs=[image_input, file_input, goal_input, duration_input, age_input, gender_input],
                outputs=[plan_output, csv_download]
            )

        # ===============================
        # TAB 2: GENERADOR DE RUTAS
        # ===============================
        with gr.TabItem("🏃 Generador de Rutas"):
            gr.Markdown("Esta herramienta te ayuda a generar rutas con las características específicas de tu plan de entrenamiento.")

            with gr.Tabs():
                # Subtab 1: Generar ruta
                with gr.TabItem("Generar Ruta"):
                    with gr.Row():
                        with gr.Column(scale=1):
                            place_input = gr.Textbox(label="Lugar de inicio", placeholder="Ej: Central Park, New York")
                            with gr.Accordion("Opciones de la ruta"):
                                distance_input = gr.Number(label="Distancia objetivo (km)", precision=2)
                                with gr.Column():
                                    gr.Markdown("--- o ---")
                                    speed_input = gr.Number(label="Velocidad media (km/h)", precision=2)
                                    time_input = gr.Number(label="Tiempo (min)", precision=2)
                                mode_input = gr.Radio(label="Tipo de entrenamiento", choices=["foot", "bike"], value="foot")
                                direction_input = gr.Dropdown(label="Dirección principal", choices=list(DIRECCIONES.keys()), value="N")
                            generate_button = gr.Button("Generar Ruta")
                            gpx_output = gr.File(label="Descargar archivo GPX")
                        with gr.Column(scale=2):
                            map_output = gr.HTML(label="Mapa de la Ruta")
                            info_output = gr.Textbox(label="Información de la Ruta", lines=3)
                            elevation_output = gr.Image(label="Perfil de Elevación")

                    generate_button.click(
                        fn=generate_route_with_gradio,
                        inputs=[place_input, distance_input, speed_input, time_input, mode_input, direction_input],
                        outputs=[map_output, gpx_output, info_output, elevation_output]
                    )

                # Subtab 2: Visualizar GPX
                with gr.TabItem("Visualizar Archivo GPX"):
                    gr.Markdown("Sube un archivo `.gpx` para visualizar la ruta en un mapa interactivo.")
                    gpx_file_upload = gr.File(label="Subir archivo GPX", type="filepath")
                    visualize_button = gr.Button("Visualizar Ruta")
                    gpx_map_output = gr.HTML(label="Mapa de la Ruta GPX")
                    gpx_info_output = gr.Textbox(label="Estado", lines=1)
                    visualize_button.click(
                        fn=visualize_uploaded_gpx,
                        inputs=[gpx_file_upload],
                        outputs=[gpx_map_output, gpx_info_output]
                    )

# Lanzar app
demo.launch(share=True, debug=True)


/usr/local/lib/python3.12/dist-packages/gradio/components/base.py:206: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4c55d6c47c9a07eb98.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4c55d6c47c9a07eb98.gradio.live
